# k8s环境搭建

## 目标

搭建一个多master节点、多worker节点集群环境，用于学习kubernets的架构（公司的kubernetes环境受限，无法使用k8s新特性），并将自己的应用服务迁移部署到k8s集群内。

## 环境准备

etcd集群至少需要3个节点，因此master高可用集群也至少需要3个节点，另外配置2个工作节点。

华为云等云服务提供商租赁虚拟机VM的成本比较高，因此在物理上通过VirtualBox虚拟化技术虚拟VM。

### VirtualBox构建虚拟VM

物理机软、硬件环境：

|资源|配置|
|---|---|
|CPU|20 Core|
|内存|64GB|
|磁盘|SSD 1T、HHD 2T|
|操作系统|Ubuntu 18.04|

网络要求：

* 虚拟机之间能互相通信
* 虚拟机能访问公网

VirtualBox的NAT Network可以满足上述要求。VirutalBox `NAT Network`配置参考[VirtualBOX NAT Network配置](https://www.cnblogs.com/ddvv/p/6882514.html)。

软件版本：

|软件|版本|
|---|---|
|VirtualBox|`6.0.24`|
|镜像|`Ubuntu 18.04`|

> **VirtualBox尽量匹配上述版本，否则，可能无法创建虚拟机，无法配置`NAT Network`网络模式。**

虚拟Ubuntu主机并配置端口转发：

1. 使用VirtualBox虚拟Ubuntu-VM1 ~ Ubuntu-VM5共5台虚拟机器，其中VM1 ~ VM3作为k8s的master节点，VM4、VM5作为k8s的工作节点
2. 配置端口转发，物理机能够将通过ssh登录到虚拟机，http、https的流量转发到虚拟机内部

实操过程省略。

### 配置虚拟机免密登录

机器列表：

`/etc/hosts`

```
10.0.2.4 Ubuntu-VM1 vm1
10.0.2.5 Ubuntu-VM2 vm2
10.0.2.6 Ubuntu-VM3 vm3
10.0.2.7 Ubuntu-VM4 vm4
10.0.2.8 Ubuntu-VM5 vm5
```

VM1作为主要的ssh登录机器，能通过ssh免密登录到其他所有机器。

1. VM安装`openssh-server`
2. 配置VM1免密登录其他所有VM

实操过程省略。

### 配置虚拟机sudo免密输入

更新`/etc/sudoers`文件

sudo vim `/etc/sudoers`

```
user_name ALL=(ALL:ALL) ALL
user_name ALL=(ALL) NOPASSWD: ALL
```

### 配置物理机登录虚拟机

通过物理机登录虚拟主要有两种方式：

1. 通过VirutalBox配置端口转发
2. 创建虚拟机到物理机ssh隧道

配置VitualBox端口转发：

实操过程省略。

虚拟机到物理机的ssh隧道：

1. 虚拟机安装autossh `sudo apt install autossh`，创建ssh隧道 `autossh -NfR 9999:localhost:22 user_name@ip`
2. 物理机安装sshpass `sudo apt install sshpass`，登录到虚拟机 `sshpass -p user_name ssh localhost -p999`

## 搭建k8s集群